# Interact with modalities connected to you Orthanc server

This notebook assumes that you have `pyorthanc` installed and that you use the `pyorthanc` repo's `docker-compose.yml` setup.
It starts a demo Orthanc server (`orthanc1`) accessible through `http://localhost:8042` with another orthanc (`orthanc2`) that will act as a modality connected to `orthanc1`.

To start the two Orthanc server:
1. Clone the `pyorthanc` repo and enter into it from your terminal
```shell
git clone https://github.com/gacou54/pyorthanc
cd pyorthanc
```
2. Start the docker compose
```shell
docker compose up orthanc1 orthanc2
```

## Creating a Orthanc client with a Modality

In [1]:
import pyorthanc

# Creating the Orthanc client
orthanc = pyorthanc.Orthanc(
    url='http://localhost:8042',  # URL of you Orthanc server
    username='orthanc',  # Using the default username
    password='orthanc',  # Using the default password
)

At this point, our PACS or second Orthanc server might not be connected to our main Orthanc server (`http://localhost:8042`) as a modality.
You can verify the existing modalities with:

In [2]:
orthanc.get_modalities()

['ORTHANC', 'my_modality']

In our example with the pyorthanc's docker-compose.yaml, `orthanc2` is still not connected as a modality.
On a production Orthanc server, the modality should be configured in the Orthanc's config file, `orthanc.json`.
In this case, we can add programmatically the modality using:

In [3]:
# Programmatically add a modality (for production use, the modality configuration should be added in the config `orthanc.json`)
orthanc.put_modalities_id('my_modality', {
    'Host': 'orthanc2',
    'Port': '4242',
    'AET': 'ORTHANC',
    'AllowEcho': True,
    'AllowFind': True,
    'AllowMove': True,
    'AllowStore': True
})

# Verify if the new modality exists
orthanc.get_modalities()

['ORTHANC', 'my_modality']

### Create a modality object
The modality object (`pyorthanc.Modality`) is the easiest way to interact with Orthanc's connected modalities.

In [4]:
my_modality = pyorthanc.Modality(orthanc, 'my_modality')

assert my_modality.echo()  # Ask Orthanc to perform a C-ECHO (i.e. a ping) to the modality, validating the connection (returns True if OK).

### Sending data from our Orthanc server to the other modality (C-STORE)
Right now, the orthanc modality doesn't have any data. We can try out to send data with the DICOM C-STORE operation!

For this, we need data in our Orthanc server:

In [5]:
from pydicom.data import get_testdata_file

small_ct_path = get_testdata_file('CT_small.dcm')

pyorthanc.upload(orthanc, small_ct_path)
series = orthanc.get_series()
assert series != []  # Ensure that we have data in our Orthanc server

Then, we can send the data from our Orthanc server to the modality (C-STORE).

In [6]:
my_modality.store(series[0])  # Here we simply send the first series that we queried above.

{'Description': 'REST API',
 'FailedInstancesCount': 0,
 'InstancesCount': 1,
 'LocalAet': 'ORTHANC',
 'ParentResources': ['93034833-163e42c3-bc9a428b-194620cf-2c5799e5'],
 'RemoteAet': 'ORTHANC'}

### Querying data (C-FIND)
A common workflow when working with an Orthanc server to another modality (such as a clinical PACS) is to find and retrieve data (C-FIND and C-MOVE in the DICOM language).

Now that we have data in out connected modality `my_modality`, can query it. We first will delete the data in our Orthanc server.

In [7]:
for series_id in orthanc.get_series():
    orthanc.delete_series_id(series_id)

assert orthanc.get_series() == []

Now, we can query the connected modality:

In [8]:
# The find function returns a query_id
query_id = my_modality.find({
    'Level': 'Series',
    'Query': {'PatientID': '*', 'Modality': 'CT'},  # You can use the typical main DICOM tag here
})

# With the query_id, you can review the answers
query_answers = my_modality.get_query_answers(query_id)
print(query_answers)

[{'AccessionNumber': '', 'Modality': 'CT', 'PatientID': '1CT1', 'QueryRetrieveLevel': 'SERIES', 'RetrieveAETitle': 'ORTHANC', 'SeriesInstanceUID': '1.3.6.1.4.1.5962.1.3.1.1.20040119072730.12322', 'SpecificCharacterSet': 'ISO_IR 100', 'StudyInstanceUID': '1.3.6.1.4.1.5962.1.2.1.20040119072730.12322'}]


### We can finally retrieve the DICOM data (C-MOVE) that we wish
After the query (C-FIND), you may want to retrieve the data. It can be done using:

In [9]:
# At this point we don't have any data in our Orthanc server
assert orthanc.get_series() == []

my_modality.move(query_id)

# We now have it once retrieve
assert orthanc.get_series() != []

# Note that you can also Move the data to a third DICOM server. Like
my_modality.move(query_id, {'TargetAET': 'ORTHANC'})

{'Description': 'REST API',
 'LocalAet': 'ORTHANC',
 'Query': [{'0008,0050': '',
   '0008,0052': 'SERIES',
   '0010,0020': '1CT1',
   '0020,000d': '1.3.6.1.4.1.5962.1.2.1.20040119072730.12322',
   '0020,000e': '1.3.6.1.4.1.5962.1.3.1.1.20040119072730.12322'}],
 'RemoteAet': 'ORTHANC'}